In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width: 98% !important; }</style>"))
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [ ]:
import sys
import os
import re
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

In [ ]:
from lib import util, embeds, fitter, masker, features, sentiment, translate

Uncomment these two lines to download the required data for the task.

In [ ]:
# !git clone https://github.com/H-TayyarMadabushi/SemEval_2022_Task2-idiomaticity.git
# !git clone https://github.com/H-TayyarMadabushi/AStitchInLanguageModels.git

In [ ]:
datapath = 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data'

Load all the CSV files in dataframes.

In [ ]:
frames = util.load_csv_dataframes(datapath)

In [ ]:
frames.keys()

In [ ]:
zdf = frames['train_zero_shot.csv']
odf = frames['train_one_shot.csv']
ddf = frames['dev.csv']
ddf_gold = frames['dev_gold.csv']
edf = frames['eval.csv']

In [ ]:
z_emb = embeds.get_embeddings(zdf)
z_emb_i = embeds.get_embeddings(zdf, append=['MWE'])

In [ ]:
multilingual_model = 'distiluse-base-multilingual-cased-v1'

### Sentence transformers embeddings

Get sentence-transformers embeddings with the best method (appending MWE to the text, ignoring context).

The "best" method isn't actually completely true, as the original paper uses the "idiomatic principle" to encode the MWE, that is, using it as a single token when tokenizing.

In [ ]:
z_emb_multi = embeds.get_embeddings(zdf, modelname=multilingual_model, append=['MWE'])

In [ ]:
d_emb_multi = embeds.get_embeddings(ddf, modelname=multilingual_model, append=['MWE'])

Do a fitting for the embeddings with Logistic Regression.

In [ ]:
z_score, z_probs, z_results = fitter.get_fit_results(z_emb_multi, zdf['Label'], d_emb_multi, ddf_gold['Label'])

In [ ]:
z_score

In [ ]:
dres = fitter.add_results(ddf, z_results, ddf_gold)

In [ ]:
dres

In [ ]:
dres_counts = util.get_counts(dres)

Show the MWEs that the model gets wrong more than half of the time. Are there any patterns?

In [ ]:
dres_counts[dres_counts['Pct correct'] < 0.5].sort_values(by=['Language','MWE'])

### Mask filling (lexical substitution)

Get several features based on mask-filling pipeline.

Rationale: It should be more difficult to get mask filling to work when the MWE is idiomatic.

There are three ways to do mask filling for the MWE:
- replace the whole expression: banana republic -\> \<mask\>
- replace the first term: \<mask\> republic
- replace the second term: banana \<mask\>

The mask filling generates several features:
- Hassub: whether a top-5 term is found in the MWE (exactly)
- Top score: the confidence score of the top term
- Short/FS/SS: Amount of "Short" terms (less than three characters) in whole mask vs first term replacement vs second term replacement, respectively

Additionally, the top terms are recorded into Top score columns.

The Top score is only recorded for an "acceptable" term (at least three characters and no non-word characters) 

In [ ]:
zdf_masked = masker.get_masked_features(zdf)

In [ ]:
zdf_masked.groupby(['Language','Label','Hassub'])['DataID'].count()

In [ ]:
ddf_masked = masker.get_masked_features(ddf)

In [ ]:
ddf_masked[ddf_masked['Hassub'] == False][535:600]

In [ ]:
ddf_masked.groupby(['Language','Hassub'])['ID'].count()

In [ ]:
str_prob = 'Além de ter sido um fracasso de bilheteria e crítica, o filme acabou marcado pelos seus efeitos especiais, principalmente ao antropomorfizar os gatos, que, bem, ficam um pouco bisonhos.'
str_prob_2 = 'Professor livre docente da Unesp, Fortaleza é presidente da Sociedade Paulista de Infectologia e membro do Comitê de Contingência da COVID-19, do Governo do Estado de São Paulo.'
str_prob_3 = 'Com a segurança da imunização em massa e os números traduzindo sua eficácia, fica mais fácil para o americano médio sentir-se confiante em marcar sua próxima viagem, gerando um circulo virtuoso para o setor nos próximos meses.'


In [ ]:
masker.replacer2(str_prob_3, 'círculo virtuoso', '<mask>', ' ')

In [ ]:
masker.replacer2(str_prob, 'efeito especial', 'efeito <mask>', ' ')

In [ ]:
masker.replace_mask_token(str_prob_2, 'livre-docente', 'livre-<mask>')

In [ ]:
masker.replace_mask_token(str_prob, 'efeito especial', 'efeito <mask>', '<mask>')

In [ ]:
masker.replace_mask_token(str_prob, 'efeito especial', '<mask>')

### Boolean features

Get features: Caps and Quotes.

Rationale:
- MWEs in Caps (Banana Republic vs banana republic) are more likely to be a proper noun (PN)
- Quoted MWEs are more likely to be idiomatic

In [ ]:
zdf_masked_feats = features.get_features(zdf_masked)

In [ ]:
zdf_masked_feats.groupby(['Language','Label','Caps'])['DataID'].count()

In [ ]:
ddf_masked_feats = features.get_features(ddf_masked)

In [ ]:
# dir(sentiment)

### Sentiment classifier

Rationale: idiomatic expressions are more likely to be affective (positive or negative).

Neutral sentiment probability is used as a proxy for literality.

In [ ]:
sentiment_classifier, sentiment_tokenizer, sentiment_config = sentiment.get_classifier_tokenizer()

In [ ]:
sentiment.get_sentiment(ddf_masked_feats['Target'].values[0], sentiment_classifier, sentiment_tokenizer, sentiment_config)

In [ ]:
ddf_masked_feats_sent = sentiment.get_df_sentiments(ddf_masked_feats, sentiment_classifier, sentiment_tokenizer, sentiment_config)

In [ ]:
zdf_masked_feats_sent = sentiment.get_df_sentiments(zdf_masked_feats, sentiment_classifier, sentiment_tokenizer, sentiment_config)

In [ ]:
zdf_masked_feats_sent[zdf_masked_feats_sent['Label'] == '0'].mean()

In [ ]:
zdf_masked_feats_sent[zdf_masked_feats_sent['Label'] == '1'].mean()

In [ ]:
btmodel1, btmodel2, bttoken1, bttoken2 = translate.get_marian_models()

### Backtranslation

Translate text from English to Portuguese and back (and vice versa if the source language is Portuguese).

Rationale: the expression is more likely to be idiomatic if it is not found from the backtranslation.

In [ ]:
zdf_bt = translate.backtranslate(zdf_masked_feats_sent, btmodel1, btmodel2, bttoken1, bttoken2, batch_len=10)

In [ ]:
ddf_bt = translate.backtranslate(ddf_masked_feats_sent, btmodel1, btmodel2, bttoken1, bttoken2, batch_len=10)

In [ ]:
# zdf_bt.sort_values(by="BT", key=lambda x: x.str.len())

In [ ]:
zdf_bt2 = translate.record_trans(zdf_bt)

In [ ]:
ddf_bt2 = translate.record_trans(ddf_bt)

In [ ]:
zdf_bt2.groupby(['Language','Label','Trans'])['DataID'].count()

In [ ]:
ddf_bt2.groupby(['Language','Trans'])['ID'].count()

### Previous/next difference

Compare the embeddings of the Target to those of Previous/Next sentence.

Rationale: Idioms are semantic outliers, thus they are more likely to be dissimilar to the context.


In [ ]:
zdf_bt3 = embeds.get_prev_next_diff(zdf_bt2, modelname=multilingual_model)
ddf_bt3 = embeds.get_prev_next_diff(ddf_bt2, modelname=multilingual_model)

In [ ]:
# zdf_bt3[zdf_bt3['Label'] == '0'].mean()
# zdf_bt3[zdf_bt3['Label'] == '1'].mean()

### Training

Combine the classifiers.

In [ ]:
zdf_t = fitter.get_trainable(zdf_bt3)
ddf_t = fitter.get_trainable(ddf_bt3)

ddf_feat_score, ddf_feat_probs, ddf_feat_results = fitter.get_fit_results(zdf_t, zdf['Label'], ddf_t, ddf_gold['Label'])
ddf_feat_score

In [ ]:
# ddf2_feat_score, ddf2_feat_probs, ddf2_feat_results = fitter.get_fit_results(zdf_t, zdf['Label'], ddf_t, ddf_gold['Label'], method='rf')
# ddf2_feat_score

# ff = fitter.check_feats(zdf_t, zdf['Label'], ddf_t, ddf_gold['Label'])
# ff.sort_values(by=['Score'], ascending=True)

In [ ]:
mu = fitter.multi_results(ddf_bt3, ddf_gold, z_results, z_probs, ddf_feat_results, ddf_feat_probs, ['Caps', 'Hassub'])
co = len(mu[mu['Prediction'] == mu['Label']])
print(co/len(mu))

In [ ]:
zdf_bt3.to_json('data/zdf_bt3.json', orient="records", lines=True)
zdf_bt3.to_csv('data/zdf_bt3.csv')

In [ ]:
ddf_bt3.to_json('data/ddf_bt3.json', orient="records", lines=True)
ddf_bt3.to_csv('data/ddf_bt3.csv')

In [ ]:
mux = fitter.multi_results(ddf_bt3, ddf_gold, z_results, z_probs, ddf_feat_results, ddf_feat_probs, ['Caps', 'Hassub'],['!Trans'])
cox = len(mux[mux['Prediction'] == mux['Label']])
print(cox/len(mux))

### One-shot model

In [ ]:
# o_emb_multi = embeds.get_embeddings(odf, modelname=multilingual_model, append=['MWE'])
# ozdf = pd.concat([zdf,odf])
# oz_emb_multi = np.concatenate([z_emb_multi, o_emb_multi])

# oz_score, oz_probs, oz_results = fitter.get_fit_results(oz_emb_multi, ozdf['Label'], d_emb_multi, ddf_gold['Label'])
# oz_score

In [ ]:
# odf_masked = masker.get_masked_features(odf)
# odf_masked_feats = features.get_features(odf_masked)
# odf_masked_feats_sent = sentiment.get_df_sentiments(odf_masked_feats, sentiment_classifier, sentiment_tokenizer, sentiment_config)
# odf_bt = translate.backtranslate(odf_masked_feats_sent, btmodel1, btmodel2, bttoken1, bttoken2, batch_len=10)
# odf_bt2 = translate.record_trans(odf_bt)
# ozdf_bt2 = pd.concat([zdf_bt2,odf_bt2])
# ozdf_t = fitter.get_trainable(ozdf_bt2)

# oddf_feat_score, oddf_feat_probs, oddf_feat_results = fitter.get_fit_results2(ozdf_t, ozdf['Label'], ddf_t, ddf_gold['Label'])
# oddf_feat_score

In [ ]:
# omu = fitter.multi_results(ddf_bt2, ddf_gold, oz_results, oz_probs, oddf_feat_results, oddf_feat_probs, ['Caps', 'Hassub'])
# co = len(omu[omu['Prediction'] == omu['Label']])
# print(co/len(omu))

### Get the best features

In [ ]:
ff2 = fitter.check_feats(zdf_t, zdf['Label'], ddf_t, ddf_gold['Label'], minfeats=5)

In [ ]:
best = fitter.get_best_features(ff2, topn=5000)

In [ ]:
bestres, bestest = fitter.get_bestest_features(best, zdf_bt3, ddf_bt3, ddf_gold, z_results, z_probs)

In [ ]:
bestres2, bestest2 = fitter.get_bestest_features(best, zdf_bt3, ddf_bt3, ddf_gold, z_results, z_probs)

In [ ]:
bestest.sort_values(by=['Score'], ascending=False)[:30]

In [ ]:
# dropcols = ['Top score', 'FS', 'SS', 'Quotes', 'MWEdiff']
dropcols = ['Hassub', 'FS', 'Nextdiff']
#dropcols = ['Top score', 'FS', 'Nextdiff', 'MWEdiff']
zdf_t4 = fitter.get_trainable(zdf_bt3).drop(dropcols, axis=1)
ddf_t4 = fitter.get_trainable(ddf_bt3).drop(dropcols, axis=1)
ddf5_feat_score, ddf5_feat_probs, ddf5_feat_results = fitter.get_fit_results(zdf_t4, zdf['Label'], ddf_t4, ddf_gold['Label'])

In [ ]:
mup = fitter.multi_results(ddf_bt3, ddf_gold, z_results, z_probs, ddf5_feat_results, ddf5_feat_probs, ['Caps', 'Hassub'],['!Trans'])
co = len(mup[mup['Prediction'] == mup['Label']])
print(co/len(mup))

### Evaluation data

In [ ]:
e_emb_multi = embeds.get_embeddings(edf, modelname=multilingual_model, append=['MWE'])

In [ ]:
ez_score, ez_probs, ez_results = fitter.get_fit_results(z_emb_multi, zdf['Label'], e_emb_multi)

In [ ]:
edf_masked = masker.get_masked_features(edf)

In [ ]:
edf_masked_feats = features.get_features(edf_masked)

In [ ]:
edf_masked_feats_sent = sentiment.get_df_sentiments(edf_masked_feats, sentiment_classifier, sentiment_tokenizer, sentiment_config)

In [ ]:
edf_bt = translate.backtranslate(edf_masked_feats_sent, btmodel1, btmodel2, bttoken1, bttoken2, batch_len=10)

In [ ]:
edf_bt2 = translate.record_trans(edf_bt)

In [ ]:
edf_bt3 = embeds.get_prev_next_diff(edf_bt2, modelname=multilingual_model)

In [ ]:
edf_bt3.to_json('data/edf_bt3.json', orient="records", lines=True)
edf_bt3.to_csv('data/edf_bt3.csv')

In [ ]:
edf_sub = frames['eval_submission_format.csv']

In [ ]:
edf_t1 = fitter.get_trainable(edf_bt3).drop(dropcols, axis=1)

edf_feat_score1, edf_feat_probs1, edf_feat_results1 = fitter.get_fit_results(zdf_t4, zdf['Label'], edf_t1)
edf_comb1 = fitter.multi_results(edf_bt3, None, ez_results, ez_probs, edf_feat_results1, edf_feat_probs1, ['Caps','Hassub'])
edf_comb1.groupby(['Language','Prediction'])['ID'].count()

In [ ]:
dropcols2 = ['Top score', 'FS', 'SS', 'Quotes', 'MWEdiff', 'Hassub', 'Caps', 'Top score 1', 'Top score 2']

zdf_et3 = fitter.get_trainable(zdf_bt3).drop(dropcols2, axis=1)
edf_et3 = fitter.get_trainable(edf_bt3).drop(dropcols2, axis=1)

edf_feat_score3, edf_feat_probs3, edf_feat_results3 = fitter.get_fit_results(zdf_et3, zdf['Label'], edf_et3)
edf_comb3 = fitter.multi_results(edf_bt3, None, ez_results, ez_probs, edf_feat_results3, edf_feat_probs3, ['Caps','Hassub'],['Quotes'])
edf_comb3.groupby(['Language','Prediction'])['ID'].count()

In [ ]:
edf_comb3[edf_comb3['Prediction'] != edf_comb1['Prediction']][['ID','MWE','Target','Hassub','Trans','Pred1','Pred2','Score1','Score2','Prediction']]

In [ ]:
dropcols3 = ['Top score', 'Hassub', 'Short', 'Nextdiff', 'MWEdiff']
zdf_et4 = fitter.get_trainable(zdf_bt3).drop(dropcols3, axis=1)
edf_et4 = fitter.get_trainable(edf_bt3).drop(dropcols3, axis=1)

edf_feat_score4, edf_feat_probs4, edf_feat_results4 = fitter.get_fit_results(zdf_et4, zdf['Label'], edf_et4)
edf_comb4 = fitter.multi_results(edf_bt3, None, ez_results, ez_probs, edf_feat_results4, edf_feat_probs4, ['Caps','Hassub'],['Quotes'])
edf_comb4.groupby(['Language','Prediction'])['ID'].count()

In [ ]:
edf_comb4[edf_comb4['Prediction'] != edf_comb3['Prediction']][['ID','MWE','Target','Hassub','Trans','Pred1','Pred2','Score1','Score2','Prediction']]

In [ ]:
edf_res2 = edf_sub.copy()
edf_res2.loc[edf_res2['Setting'] == 'zero_shot', 'Label'] = edf_comb2['Prediction']
edf_res2.to_csv('data/eval_sub_20211208_4.csv', index=False)

In [ ]:
edf_res4 = edf_sub.copy()
edf_res4.loc[edf_res4['Setting'] == 'zero_shot', 'Label'] = ez_results
edf_res4[edf_res4['Setting'] == 'zero_shot'].groupby(['Language','Label'])['ID'].count()

In [ ]:
edf_res4.to_csv('data/eval_sub_20211208_6.csv', index=False)